In [17]:
import numpy as np

x = np.array([[1, 2, 3], [4, 5, 6]], np.int32)
y = np.array([[7,8,9], [10,11,12]], np.int32)
z = np.array([[13,14,15], [16,17,18]], np.int32)
stack = np.stack((x,y,z))
stack.max().dtype

dtype('int32')

In [18]:
stack1 = stack*(1/stack.max())
stack1

array([[[0.05555556, 0.11111111, 0.16666667],
        [0.22222222, 0.27777778, 0.33333333]],

       [[0.38888889, 0.44444444, 0.5       ],
        [0.55555556, 0.61111111, 0.66666667]],

       [[0.72222222, 0.77777778, 0.83333333],
        [0.88888889, 0.94444444, 1.        ]]])

In [28]:
#stack2 = (stack - stack.mean())/stack.std()
stack2 = (stack - stack.min())/(stack.max() - stack.min())
stack2

array([[[0.        , 0.05882353, 0.11764706],
        [0.17647059, 0.23529412, 0.29411765]],

       [[0.35294118, 0.41176471, 0.47058824],
        [0.52941176, 0.58823529, 0.64705882]],

       [[0.70588235, 0.76470588, 0.82352941],
        [0.88235294, 0.94117647, 1.        ]]])

In [56]:
import geopandas as gpd
from pathlib import Path
import rasterio
import rasterstats
import numpy as np
from extract_rasters import ExtractFromTiles
BASEDIR = Path('.').resolve().parent
save_folder = Path('/home/david/Documents/programming/python/work/seasonality/viirs/datain/shps/HTI/HTI_Adm_units')
shp = BASEDIR.joinpath('datain/shps/HTI/HTI_adm1.shp')
gdf = gpd.read_file(str(shp))
gdf = gdf[['ADM1', 'ADM1_id', 'geometry']]
for row in gdf.iterrows():
    adm_df = gpd.GeoDataFrame({'ADM1': [row[1][0]], 'ADM1_id': [row[1][1]], 'geometry': [row[1][2]]})
    name = (save_folder.joinpath("{0}.shp".format(row[1]["ADM1"])))
    adm_df.to_file(name)